<a href="https://colab.research.google.com/github/tawsif67/VQA/blob/main/data_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import random

with open('input.csv', 'r') as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)  # Skip the header row
    rows = list(csv_reader)

elements_dict = {}
for row in rows:
    image_name = row[0]
    element = row[1]
    if image_name not in elements_dict:
        elements_dict[image_name] = []
    elements_dict[image_name].append(element)

question_templates = [
    "Which thing is farthest from {}?",
    "Which thing is closest to {}?",
    "Where is the {}?",
]

used_questions = {}

output_rows = []

for image_name, elements in elements_dict.items():
    available_elements = elements.copy()
    used_questions[image_name] = []
    for _ in range(5):
        if available_elements:
            element = random.choice(available_elements)
            available_elements.remove(element)

            question_template = random.choice(question_templates)

            if "Where" in question_template:
                # Filter available_elements to only include "block" or "bowl" elements
                filtered_elements = [e for e in available_elements if "block" in e or "bowl" in e]
                if filtered_elements:
                    element = random.choice(filtered_elements)
                    available_elements.remove(element)
                else:
                    continue  # Skip this question if no suitable elements are left

            question = question_template.format(element)
            used_questions[image_name].append(question)
            output_rows.append([image_name, question])

with open('questions_output.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['image', 'question'])
    csv_writer.writerows(output_rows)

In [ ]:
import pandas as pd
import re

# Define a function to extract the last two words or last three words (if the last word is "corner")
def extract_q_element(question):
    words = re.findall(r'\b\w+\b', question)
    if len(words) >= 2:
        if words[-1].lower() == "corner":
            return ' '.join(words[-3:])
        if words[-1].lower() == "middle":
            return ' '.join(words[-1:])
        else:
            return ' '.join(words[-2:])
    else:
        return ''

# Input CSV file and output CSV file names
input_csv_filename = 'questions_output.csv'
output_csv_filename = 'output.csv'

# Read the input CSV into a DataFrame
df = pd.read_csv(input_csv_filename)

# Apply the function to create a new column 'q_element'
df['q_element'] = df['question'].apply(extract_q_element)

# Save the updated DataFrame to a new CSV file
df.to_csv(output_csv_filename, index=False)

print(f"Updated CSV saved to {output_csv_filename}")

Updated CSV saved to output.csv


In [ ]:
import csv

# Read the input CSV files
with open('/content/input.csv', 'r') as f:
    csv_reader = csv.reader(f)
    input_header = next(csv_reader)
    input_rows = list(csv_reader)

with open('/content/output.csv', 'r') as f:
    csv_reader = csv.reader(f)
    questions_header = next(csv_reader)
    questions_rows = list(csv_reader)

questions_dict = {}
for row in questions_rows:
    image_name = row[0]
    question = row[1]
    q_element = row[2]  # Add a new variable to store q_element

    if image_name not in questions_dict:
        questions_dict[image_name] = []
    questions_dict[image_name].append((question, q_element))  # Store q_element along with the question

output_rows = []
for row in input_rows:
    image_name = row[0]
    element = row[1]
    coordinate = row[2]
    if image_name in questions_dict:
        questions = questions_dict[image_name]
        if questions:
            question, q_element = questions.pop(0)
            output_rows.append([image_name, question, q_element, element, coordinate])  # Include q_element in the output
        else:
            output_rows.append([image_name, '', '', element, coordinate])  # Include an empty q_element if no question
    else:
        output_rows.append([image_name, '', '', element, coordinate])  # Include an empty q_element for unmatched input rows

# Write the new CSV file
with open('combined_output.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['image', 'question', 'q_element', 'element', 'coordinate'])
    csv_writer.writerows(output_rows)

print("Combined CSV file with 'q_element' column has been created.")

Combined CSV file with 'q_element' column has been created.


In [ ]:
import csv

# Read the combined output CSV file
with open('combined_output.csv', 'r') as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    rows = list(csv_reader)

# Create a dictionary to store elements and their coordinates for each image name
element_coordinate_dict = {}
for row in rows:
    image_name = row[0]
    element = row[3]  # Get the element from the 'element' column
    coordinate = row[4]  # Get the coordinate from the 'coordinate' column
    if image_name not in element_coordinate_dict:
        element_coordinate_dict[image_name] = {}
    element_coordinate_dict[image_name][element] = coordinate

# Update the rows with q_element coordinates
for row in rows:
    q_element = row[2]  # Get the q_element from the 'q_element' column
    image_name = row[0]
    if image_name in element_coordinate_dict and q_element in element_coordinate_dict[image_name]:
        q_coordinate = element_coordinate_dict[image_name][q_element]
    else:
        q_coordinate = ''  # Set q_coordinate to an empty string if not found
    row.append(q_coordinate)  # Append q_coordinate to the row

# Update the header to include the new column
header.append('q_coordinate')

# Write the updated data to a new CSV file
with open('updated_combined_output.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(header)  # Write the updated header
    csv_writer.writerows(rows)  # Write the updated rows

print("Updated CSV file with 'q_coordinate' column has been created.")

Updated CSV file with 'q_coordinate' column has been created.


In [ ]:
import csv
import re
import math

# Function to calculate Euclidean distance between two points (coordinates)
def calculate_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Read the updated_combined_output CSV file
with open('updated_combined_output.csv', 'r') as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    rows = list(csv_reader)

# Define a list of words that indicate "closest" or "nearest"
closest_keywords = ["closest", "nearest"]
farthest_keywords = ["farthest", "furthest"]

# Define the result column header
header.append('result')

# Create a dictionary to group elements by image name
image_elements = {}
for row in rows:
    image_name = row[0]
    element = row[3]
    if image_name not in image_elements:
        image_elements[image_name] = []
    image_elements[image_name].append((element, tuple(map(int, re.findall(r'\d+', row[4])))))  # Store element and its coordinate as a tuple

# Process the rows to calculate and add the "result" column
for row in rows:
    question = row[1]
    q_coordinate = tuple(map(int, re.findall(r'\d+', row[5])))  # Extract and parse the q_coordinate as a tuple
    image_name = row[0]

    if any(keyword in question.lower() for keyword in closest_keywords):
        # Calculate the shortest distance element within the same image
        min_distance = float('inf')
        closest_element = ''
        closest_element_coord = ()
        if image_name in image_elements:
            for element, element_coordinate in image_elements[image_name]:
                if "block" in element.lower() or "bowl" in element.lower():
                    distance = calculate_distance(q_coordinate, element_coordinate)
                    if distance > 0:  # Disregard distance value 0
                        #print(f"Calculating distance between {question} ({q_coordinate}) and {element} ({element_coordinate}): {distance}")
                        if distance < min_distance:
                            min_distance = distance
                            closest_element = element
                            closest_element_coord = element_coordinate
        #print(f"Closest element for {question}: {closest_element} ({closest_element_coord})")
        row.append(closest_element if closest_element else 'N/A')  # Add the closest element or 'N/A' if no element found
    elif any(keyword in question.lower() for keyword in farthest_keywords):
        # Calculate the farthest distance element within the same image
        max_distance = 0
        farthest_element = ''
        farthest_element_coord = ()
        if image_name in image_elements:
            for element, element_coordinate in image_elements[image_name]:
                if "block" in element.lower() or "bowl" in element.lower():
                    distance = calculate_distance(q_coordinate, element_coordinate)
                    #print(f"Calculating distance between {question} ({q_coordinate}) and {element} ({element_coordinate}): {distance}")
                    if distance > max_distance:
                        max_distance = distance
                        farthest_element = element
                        farthest_element_coord = element_coordinate
        #print(f"Farthest element for {question}: {farthest_element} ({farthest_element_coord})")
        row.append(farthest_element if farthest_element else 'N/A')  # Add the farthest element or 'N/A' if no element found
    else:
        # If "Where" is in the question, set the result to N/A
        #print(f"No distance calculation needed for {question}")
        row.append('N/A')

# Write the updated data with the "result" column to a new CSV file
with open('final_output.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(header)
    csv_writer.writerows(rows)

print("Final output CSV file has been created.")

Final output CSV file has been created.


In [ ]:
import csv
import re
import math

# Function to calculate Euclidean distance between two points (coordinates)
def calculate_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Function to find the closest coordinate from a list of coordinates
def find_closest_coordinate(target_coord, coordinates):
    min_distance = float('inf')
    closest_coord = ''
    for coord_name, coord in coordinates.items():
        distance = calculate_distance(target_coord, coord)
        if distance < min_distance:
            min_distance = distance
            closest_coord = coord_name
    return closest_coord

# Read the updated_combined_output CSV file
with open('final_output.csv', 'r') as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    rows = list(csv_reader)

# Define the result column header
header.append('result1')

# Define the coordinates and their names
coordinates = {
    'bottom left corner': (0, 0),
    'bottom side': (125, 0),
    'bottom right corner': (250, 0),
    'left side': (0, 125),
    'middle': (125, 125),
    'right side': (250, 125),
    'top left corner': (0, 250),
    'top side': (125, 250),
    'top right corner': (250, 250)
}

# Process the rows to calculate and add the "result" column
for row in rows:
    question = row[1]
    q_coordinate = tuple(map(int, re.findall(r'\d+', row[5])))  # Extract and parse the q_coordinate as a tuple

    if "Where" in question:
        # Find the closest coordinate from the provided coordinates
        closest_coord = find_closest_coordinate(q_coordinate, coordinates)
        #print(f"Closest coordinate for {question}: {closest_coord}")
        row.append(closest_coord)  # Add the closest coordinate to the "result" column
    else:
        row.append('N/A')

# Write the updated data with the "result" column to a new CSV file
with open('final_output.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(header)
    csv_writer.writerows(rows)

print("Final output CSV file has been created.")

Final output CSV file has been created.


In [ ]:
import pandas as pd

# Define the input and output file paths
input_csv_file = "final_output.csv"
output_csv_file = "output.csv"

# Read the input CSV into a DataFrame
df = pd.read_csv(input_csv_file)

# Function to swap "top" and "bottom" in a given string
def swap_top_bottom(text):
    if pd.notna(text):
        return text.replace("top", "TEMP").replace("bottom", "top").replace("TEMP", "bottom")
    else:
        return text

# Apply the function to the 'question' column
df['question'] = df['question'].apply(swap_top_bottom)

# Save the modified DataFrame to a new CSV file
df.to_csv(output_csv_file, index=False)

print(f'Swapped "top" and "bottom" in {len(df)} rows and saved the result to {output_csv_file}.')

Swapped "top" and "bottom" in 97909 rows and saved the result to output.csv.


In [ ]:
import pandas as pd

# Define the input and output file paths
input_csv_file = 'output.csv'
output_csv_file = 'output.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file)

# Fill empty cells in the "result" column with the corresponding "result1" value
df['result'].fillna(df['result1'], inplace=True)

# Save the updated DataFrame as a new CSV file
df.to_csv(output_csv_file, index=False)

print(f'Updated data saved to {output_csv_file}')

Updated data saved to output.csv


In [ ]:
import pandas as pd

# Define the input and output file paths
input_csv_file = "output.csv"
output_csv_file = "final_data.csv"

# Read the input CSV into a DataFrame
df = pd.read_csv(input_csv_file)

# Task 1: Keep only the 'image', 'question', and 'result' columns
df = df[['image', 'question', 'result']]

# Task 2: Remove rows with empty 'question' column
df = df.dropna(subset=['question'])

# Task 3: Rename 'result' column to 'answer'
df = df.rename(columns={'result': 'answer'})

# Task 4: Save the modified DataFrame to a new CSV file
df.to_csv(output_csv_file, index=False)

print(f"Processed the CSV file and saved the result to {output_csv_file}.")

Processed the CSV file and saved the result to final_data.csv.


In [ ]:
import pandas as pd

# Define the input and output file paths
input_csv_file = 'final_data.csv'
output_csv_file = 'final_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file)

# Replace "img" with "image_" in the "image" column
df['image'] = df['image'].str.replace('img', 'image_')

# Save the updated DataFrame as a new CSV file
df.to_csv(output_csv_file, index=False)

print(f'Updated data saved to {output_csv_file}')

Updated data saved to final_data.csv


In [ ]:
import pandas as pd

# Define the input and output file paths
input_csv_file = 'final_data.csv'
output_csv_file = 'output.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file)

# Create a mapping of unique values in the "answer" column to unique labels
unique_answers = df['answer'].unique()
label_mapping = {answer: label + 1 for label, answer in enumerate(unique_answers)}

# Add a new column "label" based on the mapping
df['label'] = df['answer'].map(label_mapping)

# Save the updated DataFrame as a new CSV file
df.to_csv(output_csv_file, index=False)

print(f'Updated data saved to {output_csv_file}')

Updated data saved to output.csv


In [ ]:
!cp 'output.csv' '/content/drive/My Drive/'